In [2]:
import numpy as np 
import pandas as pd
import pulp
import json

variables:
I = beginning inventory
P = purchase volume
S = sales volume

Cost Function = total_sales_revenue - purchase_cost - inventory_cost

total_sales_revenue = price * sell_variable

purchase_cost = price * purchase_variable

inventory_cost = fixed_cost * average_inventory

average_inventory = I_ _n_  + 0.5 * P_ _n_ - 0.5 * S_ _n_

beginning_inventory = I_ _n-1_ + P_ _n-1_ - S_ _n-1_

ending_inventory = I_ _n_ + P_ _n_ - S_ _n_

Inventory constraint = I_ _n_ + S_ _n-1_ - P_ _n-1_ - I_ _n-1_ = 0

ending inventory constraint = 0 <= ending_inventory_ _n_ <= max_ending_inv

purchase_constraint = P_ _n_ <= purchase_max

sell_constraint = S_ _n_ <= sales_max

In [33]:
#Load the file into separate dataframes
with open('secret.json') as f:
    filepath = json.load(f)['filepath']
file_name = 'star_soybean.txt'

#read monthly soybean prices from file
soybean_file = open(filepath+file_name)
months = soybean_file.readline().split('\t')
prices = soybean_file.readline().split('\t')

#format months and prices lists
months = [month.rstrip('\n') for month in months]
prices = [float(price.strip('$ ')) for price in prices]

#define fixed variables
purchase_max = 1000.
sales_max = 2000.
beginning_inv = 470.
fixed_cost = 10.
max_ending_inv = 4000.


In [35]:
print(prices)

[110.0, 125.0, 140.0, 160.0, 165.0, 180.0, 190.0, 175.0, 155.0, 135.0, 145.0, 160.0]


In [36]:
print(months)

['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


In [138]:
#Create the model
model = pulp.LpProblem(name='soybean_planning', sense=pulp.LpMaximize)

#Create a matrix of variables for purchase qty, sell qty and beginning inventory for each month
purchase_variables = pulp.LpVariable.matrix('P', months,
                                             lowBound=0, cat=pulp.LpContinuous)
sell_variables = pulp.LpVariable.matrix('S', months,
                                             lowBound=0, cat=pulp.LpContinuous)
inventory_variables = pulp.LpVariable.matrix('I', months,
                                             lowBound=0, cat=pulp.LpContinuous)                                             

In [139]:
#set purchase constraint for each month, must be less than purchase max value
for index, month in enumerate(months):
    model.addConstraint(pulp.LpConstraint(
        e=purchase_variables[index],
        sense=pulp.LpConstraintLE,
        name='purchase_max' + month,
        rhs=purchase_max))
#set sales constraint for each month, must be less than sales max value
    model.addConstraint(pulp.LpConstraint(
        e=sell_variables[index],
        sense=pulp.LpConstraintLE,
        name='sales_max' + month,
        rhs=sales_max))
#ending inventory must be less than max ending inventory
    model.addConstraint(pulp.LpConstraint(
        e=inventory_variables[index] + purchase_variables[index] - sell_variables[index] ,
        sense=pulp.LpConstraintLE,
        name='inventory_max' + month,
        rhs=max_ending_inv))
#ending inventory must be greater than or equal to 0
    model.addConstraint(pulp.LpConstraint(
        e=inventory_variables[index] + purchase_variables[index] - sell_variables[index] ,
        sense=pulp.LpConstraintGE,
        name='inventory_min' + month,
        rhs=0))

#beginning inventory minus previous period ending inventory must be 0
for index, month in enumerate(months[1:]):
    model.addConstraint(pulp.LpConstraint(
        e=inventory_variables[index+1] + sell_variables[index] - purchase_variables[index] - inventory_variables[index],
        sense=pulp.LpConstraintEQ,
        name='inventory_constraint' + month,
        rhs=0)) 

#set beginning inventory value
model.addConstraint(pulp.LpConstraint(
    e=inventory_variables[0],
    sense=pulp.LpConstraintEQ,
    name='initial_inv',
    rhs=beginning_inv)) 

#set objective
total_sales_revenue = pulp.lpSum(sell_variables * np.array(prices))
purchase_cost = pulp.lpSum(purchase_variables * np.array(prices)) 
inventory_cost = pulp.lpSum((inventory_variables[index]+0.5*purchase_variables[index] - 0.5*sell_variables[index])*fixed_cost
                        for index, month in enumerate(months))                                

objective = total_sales_revenue - purchase_cost - inventory_cost

model.setObjective(objective)

model.solve()

#if the model is optimal, print the objective value and the production matrix
if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.0f}')
    output = pd.DataFrame()
    for index,month in enumerate(months):
        output[month]=[inventory_variables[index].value(),
                    purchase_variables[index].value(),
                    sell_variables[index].value()]

    print(output)
else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')

status: 1, Optimal
objective: $118,750
      Jan     Feb     Mar     Apr     May     Jun     Jul     Aug     Sep  \
0   470.0  1470.0  2470.0  3470.0  1470.0  2470.0   470.0     0.0     0.0   
1  1000.0  1000.0  1000.0     0.0  1000.0     0.0  1000.0  1000.0  1000.0   
2     0.0     0.0     0.0  2000.0     0.0  2000.0  1470.0  1000.0  1000.0   

      Oct     Nov     Dec  
0     0.0  1000.0  2000.0  
1  1000.0  1000.0     0.0  
2     0.0     0.0  2000.0  
